In [1]:
from mixclu import *

from IPython.display import display
pd.options.display.max_columns = None

In [2]:
df     = pd.read_csv('./datasets/church.txt')
labels = df.pop('labels')

In [ ]:
df.head(5)

In [3]:
id_col, cat_col, con_col = get_types(df)
con_col.extend(id_col)

In [4]:
scores      = pd.DataFrame()
predictions = pd.DataFrame()
predictions['Label'] = labels
data        = df

In [5]:
# fg = FAMD_2(df)

fg = famd_kemans(df, 
                cat_col, 
                clusters = 4, 
                random_state = 32, 
                max_iter = 300, 
                verbose = 0, 
                df_output = False)

NameError: name 'FAMD_2' is not defined

In [ ]:
scores = evaluate_clusters(scores,  list(yu['clusters']),
                           labels, name='FAMD_Kmeans', 
                           X=EMD)
scores

In [ ]:
yu = kmeans_model(fg, 
                 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)

In [ ]:
def kmeans_famd(df, 
                cat_columns, 
                total_clusters,
                famd_feat_no = 5,
                famd_iter    = 3,
                random_state = 42,
                kmax_iter    = 300, 
                verbose      = 0, 
                df_output    = False):
    
    """model : 4"""
    
    df_famd          = famd_feats(df, cat_columns, famd_feat_no, famd_iter)
    
    result           = kmeans_model(df              = df_famd, 
                                     no_of_clusters = total_clusters, 
                                     random_state   = random_state, 
                                     max_iter       = kmax_iter, 
                                     verbose        = verbose, 
                                     df_output      = df_output)
    return result,df_famd



res, EMD = kmeans_famd(df, 
                cat_col, 
                4,
                famd_feat_no = 9,
                famd_iter    = 300,
                random_state = 32,
                kmax_iter    = 300, 
                verbose      = 0, 
                df_output    = False)

In [ ]:
res['clusters']

In [ ]:
scores = evaluate_clusters(scores,  list(yu['clusters']),
                           labels, name='FAMD_Kmeans', 
                           X=EMD)
scores

In [ ]:

scores = evaluate_clusters(scores,  res['clusters'],
                           labels, name='FAMD_Kmeansq', 
                           X=fg)
scores

In [ ]:
def kmoid_grower(df, 
                   cat_columns, 
                   no_of_clusters, 
                   metric       = 'precomputed', 
                   method       = 'pam', 
                   init         = 'build', 
                   max_iter     = 300, 
                   random_state = 4, 
                   df_output    = False):
    
    gower_distance_matrix = calculate_gower_distance(df, cat_columns)
    model_e               = kmedoid_model(gower_distance_matrix, 
                                          no_of_clusters, 
                                          metric       = metric , 
                                          method       = method, 
                                          init         = init, 
                                          max_iter     = max_iter, 
                                          random_state = random_state, 
                                          df_output    = df_output)
    
    return model_e, gower_distance_matrix

In [ ]:
model_gro = kmoid_grower(df, 
                   cat_col, 
                   4, 
                   metric       = 'precomputed', 
                   method       = 'pam', 
                   init         = 'build', 
                   max_iter     = 300, 
                   random_state = 32, 
                   df_output    = False)

In [ ]:
model_gro[0]['clusters']
model_gro[1]

In [ ]:
def calculate_gower_distance33(df):
  
  '''
  Takes a dataframe as an input and returns the gower distance
  matrix.
  
  '''

  variable_distances = []
  
  for col in range(df.shape[1]):
    
    feature = df.iloc[:,[col]]
    if feature.dtypes.values == np.object:
      
      feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature, drop_first=True))
        
    else:
      
      feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / max(np.ptp(feature.values),1)


      variable_distances.append(feature_dist)
      

  return np.array(variable_distances).mean(0)

In [ ]:

def kmedoids_model(df):
    
    
    
    gower_data              = data.copy()
    
gower_distance_matrix   = calculate_gower_distance(gower_data)
umap_embeddings         = umap.UMAP(random_state=42, n_components=2).fit_transform(gower_distance_matrix)

gower_data['x']         = umap_embeddings[:,0]
gower_data['y']         = umap_embeddings[:,1]
gower_data['target']    = [str(x) for x in labels]

In [ ]:
num_clusters = 4
random_state = 32



# gower_data["preds"] = list(model_gro['clusters'])
# gower_data["preds"] = [str(k) for k in list(model_gro['clusters'])]

# predictions['gower_preds'] = gower_clusterer.labels_

scores = evaluate_clusters(scores, list(model_gro[0]['clusters']),
                           labels, 
                           name='gower_matrix_kmediods_pam', 
                           X=model_gro[1])

scores

In [ ]:
# design base models first

In [ ]:
# preprocessing
id_col, cat_col, con_col = get_types(df)
id_col, cat_col, con_col

In [ ]:
df.info()

In [ ]:
cat_col = ['State', "Int'l Plan", 'VMail Plan', 'Churn?']
con_col = ['Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 
           'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 
           'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 
           'Intl Charge', 'CustServ Calls']

df_pre = autopreprocessing(df, 
                  cat_col, 
                  id_columns      = None,
                  con_colmns      = con_col,
                  y               = None, 
                  allowed_missing = 20.0, 
                  corr_thr        = None)

In [ ]:
df          = df_pre[0]
con_columns = con_col
cat_columns = cat_col

In [ ]:
"""Model : 1"""

# 1. Cluster based on continuous data only


df_cos = df[con_columns]
result = kmeans_model(df_cos, 5, df_output = False)
result
# print(list(result['clusters'])[:10], y[:10])

In [ ]:
"""Model : 2"""

# 1. Cluster based on mix data with one hot encoding

one_hot_result1 = kmeans_onehot_mix(df, 
                                   cat_columns, 
                                   5, 
                                   scale = None, 
                                   df_output    = False)
print(list(one_hot_result1['clusters'])[:10], y[:10])

In [ ]:
# k prototype result

"""Model : 3"""

k_proc_result = k_prot_model(df, cat_columns, 5)
print(list(k_proc_result['clusters'])[:10], y[:10])

In [ ]:
""" Model : 4"""

y_tr = kmeans_famd(df, cat_columns, 5, 'auto')
print(list(y_tr['clusters'])[:10], y[:10])

In [ ]:
# why not just calculate mean and sd for each grouping, zscore by group, 
# and manually inspect points w abs(z score) > threshold (eg, 2, 3, etc).

# You could also calculate skew by group ((mean - median)/sd). 
# If there are outliers, you'd have large skew values.

In [ ]:
df = df_pre[0].head(5)

In [ ]:
def add_freq(df, column_name):
    df[f'{column_name}_freq'] = df[column_name].map(df[column_name].value_counts())
    return df

In [ ]:
add_freq(df, 'Churn?')

In [ ]:
from kmodes.util.dissim import ng_dissim
from kmodes.kmodes import KModes

def kmodels_model(df, init='Huang', 
                  n_init=1, 
                  verbose=2, 
                  cat_dissim = None, 
                  df_output = True):
    
    if cat_dissim:
        model = KModes(n_clusters=K,
                       init='Huang',
                       n_init=1,
                       cat_dissim=ng_dissim,
                       verbose=2)
    else:
        model = KModes(n_clusters=K,
                       init='Huang',
                       n_init=1,
                       verbose=2)
        
    clusters = model.fit_predict(df)
    
    if df_output:
        df['clusters']   = list(clusters)
        return df, model
    
    return {'clusters': clusters, 
            'model'   : model}
    

In [ ]:
pd.DataFrame(df.mean()).T

In [ ]:
m = pd.DataFrame(df.mean()).T

In [ ]:
m = pd.DataFrame(df.mean()).T
n = pd.DataFrame(df.std()).T

In [ ]:
new_cdf = pd.concat([m]*3, ignore_index=True)
new_cdf

In [ ]:
m

In [ ]:
n

In [ ]:
m/n

In [ ]:
df = pd.DataFrame({'Sex': ['M','M','M','F','F','F','F'], 'Age': [33,42,19,64,12,30,32], 'Height': ['163','167','184','164','162','158','160'],})


In [ ]:
df

In [ ]:
df['Height'] = df['Height'].astype(float)
df

In [ ]:
(33 + 42 + 19 )/3

In [ ]:
df[['Sex', 'Age', 'Height']].groupby('Sex').mean()



In [ ]:
z = df[['Sex', 'Age', 'Height']].groupby('Sex').transform(
    lambda group: (group.mean()))
z

In [ ]:
.transform(
    lambda group: (group - group.mean()).div(group.std()))
z

In [ ]:
outliers = z.abs() > stds

In [ ]:
df, cat_columns, con_feats, y = get_dummy_data(50, 
                                               8, 
                                               6,
                                               centers        = 3,
                                               missing_values = None, 
                                               id_cols        = None)
df.head(10)


In [ ]:
df

In [ ]:
['X0', 'X2', 'X4', 'X5', 'X6', 'X7']

In [ ]:
cat_columns = ['X0', 'X2', 'X4', 'X5', 'X6', 'X7']

col_1 = cat_columns[0]
col_2 = cat_columns[1]
col_3 = cat_columns[2]
col_4 = cat_columns[3]
col_5 = cat_columns[4]
col_6 = cat_columns[5]

df1 = df[(df[col_1] == 0) & (df[col_2] == 0)]
df2 = df[(df[col_1] == 1) & (df[col_3] == 1)]
df3 = df[(df[col_1] == 1) & (df[col_4] == 1)]
df4 = df[(df[col_1] == 1) & (df[col_6] == 0)]

df5 = df[(df[col_6] == 1) & (df[col_5] == 0)]
df6 = df[(df[col_2] == 1) & (df[col_4] == 0)]
df7 = df[(df[col_4] == 1) & (df[col_6] == 0)]

df8 = df[(df[col_3] == 1) & (df[col_5] == 0)]
df9 = df[(df[col_3] == 1) & (df[col_4] == 1)]

df10 = df[(df[col_5] == 1) & (df[col_2] == 0)]

# 




def add_freq(df, column_name):
    
    for col in cat_columns:
        df[f'{col}_freq'] = df[col].map(df[col].value_counts())
        df = df.drop(col, axis = 1)
    return df



all_groups  = [df1, df2, df3, df4, df5, 
              df6, df7, df8, df9, df10]

all_freq_df = [add_freq(group, cat_columns) for group in all_groups]
all_mean_df = [pd.DataFrame(group.mean()).T for group in all_freq_df] 
all_groups  = pd.concat(all_mean_df).reset_index(drop=True)
all_groups

In [ ]:
# Storring mean and std for every col as a tuple, 0 index for max value,
# and 1 for min value
outliers = []
coa      = []
for col in all_groups.columns:
    mean = np.mean(all_groups[col].values)
    std = np.std(all_groups[col].values)
    outliers.append((mean + std, mean - std))
    coa.append(col)

In [ ]:

data= all_groups['X0_freq'].values
_mean, _std = np.mean(data), np.std(data)
border = _std * 3
lower, upper = _mean - border, _mean + border
#  outliers
outliers = [x for x in data if x < lower or x > upper]
outliers

In [ ]:
list(zip(outliers,coa))

In [ ]:
all_groups

In [ ]:
from scipy import stats
all_groups[(np.abs(stats.zscore(all_groups)) < 3).all(axis=1)]

In [ ]:
def add_freq(df, column_name):
    df[f'{column_name}_freq'] = df[column_name].map(df[column_name].value_counts())
    df = df.drop(column_name, axis = 1)
    return df


# df = add_freq(df, 'X0')
# df = add_freq(df, 'X1')
# df = add_freq(df, 'X2')
# df = add_freq(df, 'X5')
# df = df.drop(['X0','X1','X3', 'X4'], axis = 1)
# df.head(5)



def gen(df, k):
    ui = []
    for i in range(k):
        dfg = df.sample(i+2)
#         dfg = add_freq(dfg, 'X0')
#         dfg = add_freq(dfg, 'X2')
#         dfg = add_freq(dfg, 'X3')
#         dfg = add_freq(dfg, 'X5')
        ui.append(dfg)
    return ui


trt = gen(df, 10)
def gene_all(dfs):
    final_d = []
    
    for k in dfs:
        final_d.append(pd.DataFrame(k.mean()).T)
    return final_d

tw = gene_all(trt)
drt= pd.concat(tw).reset_index(drop=True)
drt

In [ ]:
from scipy import stats
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

In [ ]:
from scipy import stats
z_scores = stats.zscore(drt)

z_scores

In [ ]:
abs_z_scores = np.abs(z_scores)
abs_z_scores


In [ ]:
filtered_entries = (abs_z_scores < 2).all(axis=1)
new_df = drt[filtered_entries]

print(new_df)

In [ ]:
def outlier_detect(df):
    for i in df.describe().columns:
        Q1=df.describe().at['25%',i]
        Q3=df.describe().at['75%',i]
        
        print(Q1, Q3)
        IQR=Q3 - Q1
        LTV=Q1 - 1.5 * IQR
        UTV=Q3 + 1.5 * IQR
        x=np.array(df[i])
        p=[]
        for j in x:
            if j < LTV or j>UTV:
                p.append(df[i].median())
            else:
                p.append(j)
        df[i]=p
    return df

outlier_detect(drt)

In [ ]:
m = pd.DataFrame(df.mean()).T
m

In [ ]:
n = pd.DataFrame(df.std()).T

In [ ]:
n

In [ ]:
new_cdf = pd.concat([m]*3, ignore_index=True)
new_cdf

In [ ]:
ty =  {'X0': {0: 1, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 1, 29: 1, 30: 0, 31: 1, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 1, 39: 0, 40: 1, 41: 0, 42: 1, 43: 0, 44: 0, 45: 1, 46: 1, 47: 1, 48: 1, 49: 0}, 'X1': {0: -0.037112917323895135, 1: -0.10487134240202785, 2: -1.2231079338781112, 3: -1.4422747724730558, 4: 1.1903093112171788, 5: 0.16264503017608584, 6: 0.09575885513801816, 7: -0.04065753545650327, 8: 0.9811627656097434, 9: -1.0895731715012618, 10: 1.2771663221280398, 11: 0.20642136730493899, 12: 1.4502341066082816, 13: 0.393823431298906, 14: 0.3451716634317143, 15: 0.4709902758164765, 16: 0.9982956103799087, 17: 1.189077916373609, 18: 0.9234439359961105, 19: -1.3255857892440723, 20: 1.2098373846214483, 21: -1.0264301443260604, 22: -1.2752711175444444, 23: -1.1775143284917524, 24: 0.259560479915767, 25: 0.8873566136283076, 26: 0.21516987874467863, 27: -1.1480968752611762, 28: -1.1903434754082, 29: 0.16553499639585526, 30: -0.027563846470247143, 31: 0.19474819789386086, 32: 1.5332001783034717, 33: -1.4746621814990961, 34: 0.9251147862187328, 35: 0.009242841373200278, 36: -1.4155649459675044, 37: 1.1476020465617858, 38: -1.3349528515873126, 39: 1.3090049690691499, 40: -1.0159692538569027, 41: 1.006261902461321, 42: -1.161160155994317, 43: 0.8833114074575376, 44: 1.0811966452823563, 45: -1.368200135415236, 46: -1.556580287072397, 47: -1.2006674694322674, 48: 0.13038922844618558, 49: 0.9941283827531714}, 'X2': {0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 1, 25: 0, 26: 1, 27: 0, 28: 0, 29: 1, 30: 1, 31: 1, 32: 0, 33: 0, 34: 0, 35: 1, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 1, 49: 0}, 'X3': {0: 0, 1: 0, 2: 1, 3: 1, 4: 1, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 0, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 1, 26: 0, 27: 1, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 1, 34: 1, 35: 0, 36: 1, 37: 1, 38: 1, 39: 0, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 0, 49: 1}, 'X4': {0: 1, 1: 1, 2: 1, 3: 1, 4: 0, 5: 1, 6: 1, 7: 1, 8: 0, 9: 1, 10: 0, 11: 1, 12: 0, 13: 1, 14: 1, 15: 1, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 1, 39: 0, 40: 1, 41: 0, 42: 1, 43: 0, 44: 0, 45: 1, 46: 1, 47: 1, 48: 1, 49: 0}, 'X5': {0: -1.6251996907891026, 1: -1.4952824550113089, 2: 0.5929477365851917, 3: 0.5188383985894559, 4: 0.8379329230408614, 5: -1.459754180360659, 6: -1.3954747896019781, 7: -1.4228738797414382, 8: 0.7961049502619677, 9: 0.5969844287269782, 10: 0.6254616540670719, 11: -1.1973174138607352, 12: 0.6743779844553507, 13: -1.3773048616218415, 14: -1.5502881165079259, 15: -1.410649926526345, 16: 0.966418551153225, 17: 0.8413042649713098, 18: 0.5947398261267023, 19: 0.5285211133411081, 20: 0.8154880527487283, 21: 0.685523955516477, 22: 0.7052301139466511, 23: 0.5694387744666269, 24: -1.3660759251156689, 25: 0.7376392137717523, 26: -1.2965881798979835, 27: 0.3247985508699227, 28: 0.8492845744063385, 29: -1.3631982627466268, 30: -1.5593937453283628, 31: -1.5647378670163918, 32: 0.7184017737689418, 33: 0.5401478202493889, 34: 0.8549277265014412, 35: -1.4324174459510242, 36: 0.5699907448414805, 37: 0.5278269967299144, 38: 0.6544095431196703, 39: 0.9956765313323911, 40: 0.49341021793456574, 41: 0.8777030715347666, 42: 0.5628001790223106, 43: 0.6932468790071539, 44: 0.5944907552098264, 45: 0.6628094310909329, 46: 0.660678722318602, 47: 0.68454503898171, 48: -1.5961965190965848, 49: 0.7606527604851616}, 'X6': {0: 0, 1: 0, 2: 0, 3: 0, 4: 1, 5: 0, 6: 0, 7: 0, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 1, 17: 1, 18: 1, 19: 0, 20: 1, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 1, 40: 0, 41: 1, 42: 0, 43: 1, 44: 1, 45: 0, 46: 0, 47: 0, 48: 0, 49: 1}, 'X7': {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 1, 22: 0, 23: 1, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 1, 37: 0, 38: 1, 39: 0, 40: 1, 41: 0, 42: 0, 43: 0, 44: 0, 45: 1, 46: 0, 47: 1, 48: 0, 49: 0}}


In [ ]:
df_mran

In [ ]:
from scipy.stats import pearsonr
df= all_groups.drop
column_values = [column for column in df.columns.tolist() ]


df['Correlation_coefficent'], df['P-value'] = zip(*df.T.apply(lambda x: pearsonr(x[column_values ],x[column_values ])))
df_result = df[['Correlation_coefficent','P-value']]

In [ ]:
all_groups

In [ ]:
from sklearn.neighbors import DistanceMetric
import pandas as pd
import numpy as np

def calculate_gower_distance(df):
  
  '''
  Takes a dataframe as an input and returns the gower distance
  matrix.
  
  '''

  variable_distances = []
  
  for col in range(df.shape[1]):
    
    feature = df.iloc[:,[col]]
    if feature.dtypes.values == np.object:
      
      feature_dist = DistanceMetric.get_metric('dice').pairwise(pd.get_dummies(feature, drop_first=True))
        
    else:
      
      feature_dist = DistanceMetric.get_metric('manhattan').pairwise(feature) / max(np.ptp(feature.values),1)


      variable_distances.append(feature_dist)
      

  return np.array(variable_distances).mean(0)

In [ ]:
calculate_gower_distance(df)

In [ ]:
import gower
gower.gower_matrix(df)

In [ ]:
import math 
def calculate_zscore(df, columns):
  '''
  scales columns in dataframe using z-score
  '''
  df = df.copy()
  for col in columns:
      df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0)

  return df



def one_hot_encode(df, columns):
  '''
  one hot encodes list of columns and
  concatenates them to the original df
  '''

  concat_df = pd.concat([pd.get_dummies(df[col], drop_first=True, prefix=col) for col in columns], axis=1)
  one_hot_cols = concat_df.columns

  return concat_df, one_hot_cols



def normalize_column_modality(df, columns):
  '''
  divides each column by the probability μₘ of the modality 
  (number of ones in the column divided by N) only for one hot columns
  '''

  length = len(df)
  for col in columns:
    
    weight = math.sqrt(sum(df[col])/length)
    df[col] = df[col]/weight

  return df



def center_columns(df, columns):
  '''
  center columns by subtracting the mean value
  '''
  for col in columns:
      df[col] = (df[col] - df[col].mean())

  return df



def FAMD_(df, n_components=2):
  '''
  Factorial Analysis of Mixed Data (FAMD), 
  which generalizes the Principal Component Analysis (PCA) 
  algorithm to datasets containing numerical and categorical variables

  a) For the numerical variables
    - Standard scale (= get the z-score)
  
  b) For the categorical variables:
    - Get the one-hot encoded columns
    - Divide each column by the square root of its probability sqrt(μₘ)
    - Center the columns

  c) Apply a PCA algorithm over the table obtained!

  '''
  
  variable_distances = []

  numeric_cols = df.select_dtypes(include=np.number)
  cat_cols = df.select_dtypes(include='object')
  
  # numeric process
  normalized_df = calculate_zscore(df, numeric_cols)
  normalized_df = normalized_df[numeric_cols.columns]

  # categorical process
  cat_one_hot_df, one_hot_cols = one_hot_encode(df, cat_cols)
  cat_one_hot_norm_df = normalize_column_modality(cat_one_hot_df, one_hot_cols)
  cat_one_hot_norm_center_df = center_columns(cat_one_hot_norm_df, one_hot_cols)

  # Merge DataFrames
  processed_df = pd.concat([normalized_df, cat_one_hot_norm_center_df], axis=1)

  # Perform (PCA)
  pca = PCA(n_components=n_components)
  principalComponents = pca.fit_transform(processed_df)

  return principalComponents


df, cat_columns, con_feats, y = get_dummy_data(50, 
                                               8, 
                                               6,
                                               centers        = 3,
                                               missing_values = None, 
                                               id_cols        = None)
df.head(10)
df = cobj(df, cat_columns)

In [ ]:
FAMD_(df, n_components=2)

In [ ]:
df.dtypes

In [ ]:
famd_feats(df, cat_columns,
                   no_of_components = 2)

In [ ]:
dat = umap_reduce(df)
result, emd = dat

UMAP_clusterer = kmeans_model(emd, 
                 no_of_clusters = 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)


def umap_embd_model(df, 
                    cat_columns, 
                    no_of_clusters, 
                    random_state = 32, 
                    max_iter     = 300, 
                    verbose      = 0, 
                    df_output    = False):
    
    
    df             = cobj(df, cat_columns)
    _, emd         = dat
    
    UMAP_clusterer = kmeans_model(emd, 
                                 no_of_clusters = no_of_clusters, 
                                 random_state = random_state, 
                                 max_iter     = max_iter, 
                                 verbose      = verbose, 
                                 df_output    = df_output)
    
    return UMAP_clusterer

In [ ]:
result, emd = dat

In [ ]:
# UMAP_clusterer = KMeans(n_clusters=num_clusters, init='k-means++',
#                         verbose=0, random_state=random_state,
#                         algorithm='auto').fit(embedding_)

In [ ]:
UMAP_clusterer = kmeans_model(emd, 
                 no_of_clusters = 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)



In [ ]:
scores = evaluate_clusters(scores,  list(UMAP_clusterer['clusters']),
                           labels, name='ump', 
                           X=emd)
scores

In [ ]:
continous_data = df.copy()

numeric_cols = continous_data.select_dtypes(include=np.number)
cat_cols = continous_data.select_dtypes(include='object')
  
# numeric process
normalized_df = calculate_zscore(continous_data, numeric_cols)
normalized_df = normalized_df[numeric_cols.columns]

# categorical process
cat_one_hot_df, one_hot_cols = one_hot_encode(continous_data, cat_cols)
cat_one_hot_norm_df = calculate_zscore(cat_one_hot_df, one_hot_cols)

# Merge DataFrames
processed_df = pd.concat([normalized_df, cat_one_hot_norm_df], axis=1)

In [ ]:
processed_df

In [ ]:
UMAP_clusterer2 = kmeans_model(processed_df, 
                 no_of_clusters = 4, 
                 random_state = 32, 
                 max_iter     = 300, 
                 verbose      = 0, 
                 df_output    = False)

In [ ]:
scores = evaluate_clusters(scores,  list(UMAP_clusterer2['clusters']),
                           labels, name='cat2enc', 
                           X=processed_df)
scores

In [ ]:
k_protype_data = df.copy()

def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

numeric_cols = k_protype_data.select_dtypes(include=np.number)
cat_cols = k_protype_data.select_dtypes(include='object')

norm_num_cols = calculate_zscore(numeric_cols, numeric_cols)
processed_df = pd.concat([norm_num_cols, cat_cols], axis=1)

categorical_indices = column_index(processed_df, cat_cols.columns)

In [ ]:
mk = k_prot_model(processed_df, 
           list(cat_cols.columns), 
           4, 
           init_method  = 'Cao',
           random_state = 32,
            max_iter    = 300,
            n_jobs       = 1,
            verbose     = 0, 
            df_output   = False, 
            n_init      = 10)

In [ ]:
scores1 = evaluate_clusters(scores,  list(mk["clusters"]),
                           labels, name='K_Prototype', 
                           X=None)

scores1